## Manipulating and Modeling Time Series Data

### Data Set Up

Airline Passenger Data: https://www.kaggle.com/rakannimer/air-passengers

(it's a pretty common dataset, available in several different places, but here's a source)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller

import pmdarima as pm
from pmdarima import model_selection
from pmdarima.utils import decomposed_plot
from pmdarima.arima import decompose

In [ ]:
# If you want to install pmdarima (plus a link to the documentation)
# http://alkaline-ml.com/pmdarima/
# !pip install pmdarima

In [ ]:
# Read df
air_df = pd.read_csv('data/airline_passengers.csv')

In [ ]:
# Check the shape of the data
air_df.shape

In [ ]:
# Check the first 10 rows...
air_df.head(10)

In [ ]:
# Let's go ahead and set the index to be a datetime index
# First making it a datetime object
air_df['Month'] = pd.to_datetime(air_df['Month'])

# Now making it our index
air_df.set_index('Month', inplace=True)

In [ ]:
# Now let's look at how that impacted the time/index
air_df.head()

In [ ]:
air_df.tail()

In [ ]:
# Let's get a sense of the data
plt.figure(figsize=(16,6))
air_df['Thousands of Passengers'].plot()
plt.title('Thousands of Passengers')
plt.show()

Now we can see the general trends in our data.

Also! This is why we change our data to use the datetime object as the index - makes EVERYTHING easier!

## Changing the frequency of our data 

Also called downsampling or upsampling, depending on whether you're going to a less frequent or more frequent point in time.

[Here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling) is a reference for resampling based on time frequency. (you can find the actual codes you can use as arguments in the resample function [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)).

In [ ]:
# Upsampling to a daily cadence
df_daily = air_df.resample('D').mean()

In [ ]:
df_daily.head()

In [ ]:
# Ew - nulls!
df_daily.isna().sum()

Of course we have nulls though - we took monthly data and tried to make it daily!

But what about downsampling to quarterly?

In [ ]:
# Here we're downsampling to quarter
# Note that there are a few different ways to define 'quarter'
df_quarterly = air_df.resample('Q').mean()
df_quarterly.head()

In [ ]:
# Again, visualizing the Average Opening price
plt.figure(figsize=(16,6))
df_quarterly['Thousands of Passengers'].plot()
plt.title('Thousands of Passengers - Quarterly')
plt.show()

So, as would make sense, as you change the frequency of your data it changes the granularity (level of detail) that's conveyed.

### Stationarity

Introduction to stationarity from [_Forecasting: Principles and Practice_](https://otexts.com/fpp2/stationarity.html):

> "A stationary time series is one whose properties do not depend on the time at which the series is observed.14 Thus, time series with trends, or with seasonality, are not stationary — the trend and seasonality will affect the value of the time series at different times. On the other hand, a white noise series is stationary — it does not matter when you observe it, it should look much the same at any point in time.
>
> "Some cases can be confusing — a time series with cyclic behaviour (but with no trend or seasonality) is stationary. This is because the cycles are not of a fixed length, so before we observe the series we cannot be sure where the peaks and troughs of the cycles will be.
>
> "In general, a stationary time series will have no predictable patterns in the long-term. Time plots will show the series to be roughly horizontal (although some cyclic behaviour is possible), with constant variance."

And here's a [useful blog post](https://towardsdatascience.com/stationarity-in-time-series-analysis-90c94f27322) on the subject, where I found the below demonstrative image:

![Examples of stationary and non-stationary processes, from the above medium blog](https://miro.medium.com/max/1400/1*tkx0_wwQ2JT7pSlTeg4yzg.png)

#### Why do we want to get a stationary series?

> "Stationarity means that the statistical properties of a time series (or rather the process generating it) do not change over time."

- [Source](https://towardsdatascience.com/stationarity-in-time-series-analysis-90c94f27322)

Biggest reason: makes the data easier to model!

In [ ]:
# Let's  get the rolling mean and rolling standard deviation, using a 12-month window

roll_mean = air_df['Thousands of Passengers'].rolling(window=12, center=False).mean()
roll_std = air_df['Thousands of Passengers'].rolling(window=12, center=False).std()

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))
ax.plot(air_df['Thousands of Passengers'], color='blue',
        label='Average Monthly Number of Passengers (Thousands)')

ax.plot(roll_mean, color='red', label='Rolling 3-Month Mean')

ax.plot(roll_mean + roll_std, color='black', linestyle='dotted')
ax.plot(roll_mean - roll_std, color='black', linestyle='dotted')

ax.legend()
fig.tight_layout()

Do you think this data is stationary? Why or why not?

 - 
 

There's a test for this!

> **Augumented Dickey-Fuller test**: a hypothesis test, where we reject the null hypothesis (that a time series is non-stationary) if the test-statistic is less than the critical value

[Documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html) for the Dickey-Fuller test in StatsModels

So let's write out our null and alternative hypotheses (remember these??):

Ho = 

Ha = 

In [ ]:
# Feed in our column, since the test function expects a series:
adfuller(air_df['Thousands of Passengers'])

Let's interpret the output of this test:

- 


In [ ]:
# Now that we've determined whether the data is stationary, let's decompose it

# Using the decompose function from pmdarima
# Need to feed it an array, hence the .values attribute
decomposed = decompose(air_df['Thousands of Passengers'].values, 'multiplicative', m=12)

# M? See this: https://alkaline-ml.com/pmdarima/tips_and_tricks.html#period

In [ ]:
# Plotting the pieces using the arima model again
decomposed_plot(decomposed, figure_kwargs={'figsize': (16, 10)})
plt.show()

From the documentation for `pmdarima`'s decompose:: 

> So what is happening when we call `decomposed`?
> 1) The trend is extracted from the signal via a convolution using either a
   SMA or a user-defined filter.
>   
> 2) We remove the effects of the trend from the original signal by either
   subtracting its effects or dividing out its effects for 'additive' or
   'multiplicative' types of decompositions, respectively. We then take the
   mean across all seasons to get the values for a single season. For m=4, we
   expect 4 values for a single season.
>
> 3) We then create the seasonal series by replicating the single season
   until it is the same length of the trend signal.
>
> 4) Lastly to get the random/noise elements of the signal we remove the effects
   of both the trend and seasonal series and we are now left with the
   variation of the original signal that is neither explainable by seasonal
   nor trend effects.
>
> This logic produces a named tuple of the original signal, trend, seasonal, and random components. It is this named tuple that is passed to `decomposed_plot`

#### Additive vs Multiplicative?

It's in the trends of the plots: 

| Additive Example | Multiplicative Example |
| ---------------- | ---------------------- |
| ![from the pmdarima documentation: additive example](images/sphx_glr_example_seasonal_decomposition_001.png) | ![from the pmdarima documentation: multiplicative example](images/sphx_glr_example_seasonal_decomposition_002.png) |

[Source](https://alkaline-ml.com/pmdarima/auto_examples/arima/example_seasonal_decomposition.html)

Can you spot the difference?

## Now - Time to Model!

In [ ]:
# New dataset who dis
# Monthly Google search trends for 'taxes' in the US
df_taxes = pd.read_csv('data/google-trends_taxes_us.csv')

# Some quick clean up
df_taxes.columns = ['counts']
df_taxes = df_taxes.iloc[1:]
df_taxes['counts'] = df_taxes['counts'].str.replace('<1', '0').astype(int)
df_taxes.index = pd.to_datetime(df_taxes.index)

In [ ]:
df_taxes.head()

In [ ]:
df_taxes.tail()

In [ ]:
df_taxes.plot(figsize=(10,6))

In [ ]:
# If we check out the index, you see freq=None
df_taxes.index

In [ ]:
# Let's set that freq to be MS (month start)
df_taxes.index.freq = 'MS'
df_taxes.index

In [ ]:
# Let's decompose this one

decomposed = decompose(df_taxes['counts'].values, 'additive', m=12)

decomposed_plot(decomposed, figure_kwargs={'figsize': (16, 10)})
plt.show()

## ARMA Modeling 

Once you determine if your time series is stationary, you can model. There are 4 key steps: 
1. Model Identification - where you determine the properties of a time series then chose a structural form. Remember you're treating the data as a series of random variables. The basic types of ARIMA models are: 
    - AutoRegressive(AR)
    - Moving Average(MA) 
    - AutoRegressive Moving Average(ARMA)
    - AutoRegressive Integrated Moving Average(ARIMA)

A time series may be primarily an autoregressive, moving average or combination of both. To identify which it is, you need to plot 2 key functions.

    > Sample Autocorrelation Function(ACF) 
    > Sample Partial Autocorrelation Function(Partial ACF)
    
2. Parameter Estimation - Once you have identified the form of an ARIMA model, the next step is to estimate the coefficients or parameters of the model. You can use Regression and MLE to do this. 

3. Model Checking - The most widely used information criterion(checking the quality of your model) for time series is AIC. You can compare different models with different numbers of lagged terms, white noise terms and how many times the time series was differenced and choose the model with the lowest AIC. 

4. Forecasting - Once the model is estimated you can forecast future values with the predict function. 

![](https://www.statisticshowto.com/wp-content/uploads/2016/11/lag-plot-linear.png)

## Autocorrelation and Partial Autocorrelation
The ACF shows the correlations between the elements of a time series as a function of their lags. The partial ACF shows the correlations between the elements of a time series for each lag, holding constant the impact of all other lags.

### Autocorrelation Function Plots

https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/

> "The **autocorrelation function** is a function that represents autocorrelation of a time series as a function of the time lag."

The autocorrelation function tells interesting stories about trends and seasonality. For example, if the original time series repeats itself every five days, you would expect to see a spike in the autocorrelation function at 5 days.



In [ ]:
df_taxes['counts']

In [ ]:
df_taxes['lag 1'] = df_taxes['counts'].shift(1)

In [ ]:
df_taxes['lag 12'] = df_taxes['counts'].shift(12)

In [ ]:
df_taxes.head()

In [ ]:
df_taxes.corr()

In [ ]:
import statsmodels.graphics.tsaplots as tsa

tsa.plot_acf(df_taxes['counts'],lags=52);

In [ ]:
# Another view
plt.figure(figsize=(20, 4))
pd.plotting.autocorrelation_plot(df_taxes['counts']);

The horizontal bands represent condfidence intervals, which are calculated by taking relevant z-scores of the standard normal distribution and dividing by the square root of the number of observations. What do these intervals mean? - anything outside confidence interval means not due to chance - reject null. 


### Partial-Autocorrelation Function Plot


> "The **partial autocorrelation function** can be interpreted as a regression of the series against its past lags.
 
 > It helps you come up with a possible order for the auto regressive term. The terms can be interpreted the same way as a standard linear regression, that is the contribution of a change in that particular lag while holding others constant. "

The idea behind Partial Autocorrelation is to compare a series to a lagged version of itself while abstracting away from intermediate values. In effect, this amounts to exploring the correlations among residuals

In [ ]:
tsa.plot_pacf(df_taxes['counts'],lags=50);

# ARIMA MODELS:


## The ARIMA Time Series Model

One of the most common methods used in time series forecasting is known as the ARIMA model, which stands for **AutoregRessive Integrated Moving Average**. ARIMA is a model that can be fitted to time series data in order to better understand or predict future points in the series.

Let's have a quick introduction to ARIMA. The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:

### Number of AR (Auto-Regressive) terms (p): 

`p` is the auto-regressive part of the model. It allows us to incorporate the effect of past values into our model. Intuitively, this would be similar to stating that it is likely to rain tomorrow if it has been raining for past 3 days. AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).

### Number of Differences (d):

`d` is the **Integrated** component of an ARIMA model. This value is concerned with the amount of differencing as it identifies the number of lag values to subtract from the current observation. Intuitively, this would be similar to stating that it is likely to rain tomorrow if the difference in amount of rain in the last *n* days is small. 

### Number of MA (Moving Average) terms (q): 

`q` is the moving average part of the model which is used to set the error of the model as a linear combination of the error values observed at previous time points in the past. MA terms form lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where `e(i)` is the difference between the moving average at ith instant and actual value.

These three distinct integer values, (p, d, q), are used to parametrize ARIMA models. Because of that, ARIMA models are denoted with the notation `ARIMA(p, d, q)`. Together these three parameters account for seasonality, trend, and noise in datasets:

* `(p, d, q)` are the non-seasonal parameters described above.
* `(P, D, Q)` follow the same definition but are applied to the seasonal component of the time series. 
* The term `s` is the periodicity of the time series (4 for quarterly periods, 12 for yearly periods, etc.).

A detailed article on these parameters is available [HERE](https://www.quantstart.com/articles/Autoregressive-Integrated-Moving-Average-ARIMA-p-d-q-Models-for-Time-Series-Analysis).

The seasonal ARIMA method can appear daunting because of the multiple tuning parameters involved. In the next section, we will describe how to automate the process of identifying the optimal set of parameters for the seasonal ARIMA time series model.

What you generally will try to do for any time series analysis is:

- Detrend your time series using differencing. ARMA models represent stationary processes, so we have to make sure there are no trends in our time series
- Look at ACF and PACF of the time series
- Decide on the AR, MA, and order of these models
- Fit the model to get the correct parameters and use for prediction

In [ ]:
# metrics
from sklearn import metrics

# Note - we're back to regression metrics!
def report_metrics(y_true, y_pred):
    print("Explained Variance:\n\t", metrics.explained_variance_score(y_true, y_pred))
    print("MAE:\n\t", metrics.mean_absolute_error(y_true, y_pred))
    print("RMSE:\n\t", metrics.mean_squared_error(y_true, y_pred, squared=False))
    print("r^2:\n\t", metrics.r2_score(y_true, y_pred))

In [ ]:
# Getting rid of those earlier lag cols
df_taxes = df_taxes.drop(columns=['lag 1', 'lag 12'])

In [ ]:
# Back to taxes
# Let's create a time period tracker - incrementing each instance
df_taxes.insert(0, 't', range(len(df_taxes)))
df_taxes.head()

In [ ]:
# Let's create a target area
df_taxes['future'] = (df_taxes.index.year > 2017).astype('int')

# Now plot
plt.figure(figsize=(10,6))
df_taxes.loc[df_taxes.future == 0, 'counts'].plot()
df_taxes.loc[df_taxes.future == 1, 'counts'].plot()
plt.show()

In [ ]:
# Here, just using the instance as our "X" - not really what we're using to predict
# But - this is setting us up for something later (linear predictions)
X_train = df_taxes.loc[df_taxes.future == 0, 't'].values.reshape(-1, 1)
X_test = df_taxes.loc[df_taxes.future == 1, 't'].values.reshape(-1, 1)
# Our train set is our actual value in the series
y_train = df_taxes.loc[df_taxes.future == 0, 'counts'].values
y_test = df_taxes.loc[df_taxes.future == 1, 'counts'].values

#### What's the most naive way to make a prediction?

- 


In [ ]:
y_pred = None

In [ ]:
# Plot your naive predictions
plt.figure(figsize=(10,6))
df_taxes.loc[df_taxes.future == 0, 'counts'].plot()
df_taxes.loc[df_taxes.future == 1, 'counts'].plot()

# Predictions go here



plt.show()

In [ ]:
# How right are we?
report_metrics(y_test, y_pred)

Thoughts? 

- 


#### Another Naive Approach - Linear Trend

In [ ]:
# linear trend approach
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_trend = lr.predict(X_train)

In [ ]:
y_pred

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df_taxes['counts'])
plt.plot(df_taxes.loc[df_taxes.future == 0].index, y_trend)
plt.plot(df_taxes.loc[df_taxes.future == 1].index, y_pred, color='green')
plt.show()

In [ ]:
report_metrics(y_test, y_pred)

Thoughts? 

- 


### ARIMA!

In [ ]:
# Import ARMA
from statsmodels.tsa.arima_model import ARIMA

# Fit an ARMA model - what should our parameters be?
arima_order = None #should be a tuple, like (0,0,0)
mod_arima = ARIMA(df_taxes.loc[df_taxes.future == 0]['counts'], 
                 order=arma_order)
res_arima = mod_arma.fit()

# Print out summary information on the fit
print(res_arima.summary())

In [ ]:
# Now, to make more predictions, how many steps do we want?
len(y_test)

In [ ]:
# Returns 3 things - forecast, standard error, and confidence interval
res_arima.forecast(steps = len(y_test))[0]

In [ ]:
# Saving these predictions to more easily visualize
future_df = df_taxes.loc[df_taxes.future == 1].copy()
future_df['preds'] = res_arima.forecast(steps = len(y_test))[0]
future_df

In [ ]:
# Visualize it!
plt.figure(figsize=(10,6))
plt.plot(df_taxes.loc[df_taxes.future == 0, 'counts'], color='blue', label='actual train')
plt.plot(df_taxes.loc[df_taxes.future == 1, 'counts'], color='orange', label='actual test')
plt.plot(future_df['preds'], color='green', label='predicted test')

Thoughts? 

- 


### PMDArima - Using their Auto ARIMA! 

Basically, grid search for ARIMA

In [ ]:
# Train test split - but now using PMDArima's function
train, test = model_selection.train_test_split(df_taxes['counts'], test_size=17)

In [ ]:
train.head()

In [ ]:
# checking stationarity
from pmdarima.arima.stationarity import ADFTest

# beyond statsmodels
adf_test = ADFTest(alpha=0.05)
p_val, should_diff = adf_test.should_diff(df_taxes['counts'])  # (0.01, False)

print(f"P-Value: {p_val}, so should you difference the data? {should_diff}")

In [ ]:
# time to model!
arima = pm.auto_arima(train, error_action='ignore', trace=True,
                      suppress_warnings=True, maxiter=100,
                      seasonal=True, m=12) 

In [ ]:
# check the output summary
arima.summary()

In [ ]:
y_pred = arima.predict(n_periods=test.shape[0])

In [ ]:
# Plot actual test vs. forecasts:
x = np.arange(test.shape[0])
plt.plot(x, test, color='orange', label='Actual')
plt.plot(x, arima.predict(n_periods=test.shape[0]), color='green', label='Predicted')
plt.title('Actual test samples vs. forecasts')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df_taxes['counts'], label = 'actual')
plt.plot(df_taxes.loc[df_taxes.future == 1].index, y_pred, color='green', label = 'predicted')
plt.legend()
plt.show()

In [ ]:
report_metrics(y_test, y_pred)

Thoughts?

- 


More from the documentation: https://alkaline-ml.com/pmdarima/tips_and_tricks.html#understand-p-d-and-q

> ARIMA models are made up of three different terms:
> 
> p: The order of the auto-regressive (AR) model (i.e., the number of lag observations). A time series is considered AR when previous values in the time series are very predictive of later values. An AR process will show a very gradual decrease in the ACF plot.
> 
> d: The degree of differencing.
> 
> q: The order of the moving average (MA) model. This is essentially the size of the “window” function over your time series data. An MA process is a linear combination of past errors.

OR:

> Seasonal ARIMA models have three parameters that heavily resemble our p, d and q parameters:
> 
> P: The order of the seasonal component for the auto-regressive (AR) model.
> 
> D: The integration order of the seasonal process.
> 
> Q: The order of the seasonal component of the moving average (MA) model.
> 
> P and Q and be estimated similarly to p and q via auto_arima, and D can be estimated via a Canova-Hansen test, however m generally requires subject matter knowledge of the data.

More gems at the end for those digging back into this notebook:

`pmdarima` has a set of tips and tricks: https://alkaline-ml.com/pmdarima/tips_and_tricks.html

Also:

- https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd
- https://machinelearningmastery.com/develop-arch-and-garch-models-for-time-series-forecasting-in-python/

And, what I'm really looking into right now:
- https://towardsdatascience.com/sktime-a-unified-python-library-for-time-series-machine-learning-3c103c139a55